In [1]:
import numpy as np
import scipy
import scipy.linalg
from numpy.linalg import eig

In [2]:
def is_lu_available(A):
    for i in range(0, len(A)):
        if np.linalg.det(A[:i, :i]) == 0:
            return False
    return True

def tmp_sum(l, u, i, j, tp):
    ret = 0
    if tp == 1:
        for k in range(0, i):
            ret += l[i][k] * u[k][j]
    else:
        for k in range(0, j):
            ret += l[i][k] * u[k][j]
    return ret

def get_lu(A):
    l = np.zeros((len(A), len(A)))
    u = np.zeros((len(A), len(A)))
    for i in range(0, len(A)):
        l[i][i] = 1
    for i in range(0, len(A)):
        for j in range(0, len(A)):
            if i <= j:
                u[i][j] = A[i][j] - tmp_sum(l, u, i, j, 1)
            else:
                l[i][j] = (A[i][j] - tmp_sum(l, u, i, j, 2)) / u[j][j]
    return l, u

def find_max_el(A, beg):
    max_row = beg
    max_col = beg
    for i in range(beg, len(A)):
        for j in range(beg, len(A)):
            if (abs(A[i][j]) > abs(A[max_row][max_col])):
                max_row = i
                max_col = j
    return max_row, max_col

def swap_rows_cols(A, max_row, max_col, ind):
    A[:, [ind, max_col]] = A[:, [max_col, ind]]
    A[[ind, max_row]] = A[[max_row, ind]]
    return A

def get_array_of_eigen(quadr_matix):
    w, v = eig(quadr_matix)
    return w

def find_max_eigen(quadr_matrix):
    arr = get_array_of_eigen(quadr_matrix)
    val = arr[0]
    if (len(arr) == 0):
        return val
    for i in range (0, len(arr)):
        var = arr[i]
        if val < var:
            val = var
    return val

def find_min_eigen(quadr_matrix):
    arr = get_array_of_eigen(quadr_matrix)
    val = arr[0]
    if (len(arr) == 0):
        return val
    for i in range (0, len(arr)):
        var = arr[i]
        if val > var:
            val = var
    return val

precision = 1e-6

### Matrix initialization ###
#### Выбрана система г) ####

In [3]:
A = np.zeros((100, 100))
b = np.zeros((100, 1))

for i in range(0, 100):
    A[0][i] = 1.0
    b[i] = 100.0 - i

for i in range(1, len(A) - 1):
    A[i][i - 1] = 1.0
    A[i][i] = 10.0
    A[i][i + 1] = 1.0

A[len(A) - 1][len(A) - 2] = 1.0
A[len(A) - 1][len(A) - 1] = 1.0


### LU or Gauss ###

In [4]:
def get_l_solution(l, b):
    y = np.zeros((len(l), 1))
    assert l[0][0] != 0
    y[0] = b[0] / l[0][0]
    for i in range(1, len(l)):
        cumulative_sum = 0
        for j in range(0, i):
            cumulative_sum += l[i][j] * y[j]
        assert l[i][i] != 0
        y[i] = (b[i] - cumulative_sum) / l[i][i]
    return y

def get_u_solution(u, y):
    x = np.zeros((len(u), 1))
    assert u[len(u) - 1][len(u) - 1] != 0
    x[len(u) - 1] = y[len(u) - 1] / u[len(u) - 1][len(u) - 1]
    for i in range(len(u) - 1, -1, -1):
        cumulative_sum = 0
        for j in range(i + 1, len(u)):
            cumulative_sum += u[i][j] * x[j]
        assert u[i][i] != 0
        x[i] = (y[i] - cumulative_sum) / u[i][i]
    return x

if (is_lu_available(A)):
    l, u = get_lu(A)
    y = get_l_solution(l, b)
    ans = get_u_solution(u, y)
    print(np.matmul(A, ans) - b)
else: #Gauss
    A_0 = np.copy(A)
    b_0 = np.copy(b)
    x_order = np.arange(0, len(A), 1)
    for i in range(0, len(A)):
        max_row, max_col = find_max_el(A, i)
        A = swap_rows_cols(A, max_row, max_col, i)
        x_order[[i, max_col]] = x_order[[max_col, i]]
        b[[i, max_row]] = b[[max_row, i]]
        for k in range(0, len(A)):
            if k != i:
                m = -1 * (A[k][i] / A[i][i])
                A[k][i] = 0
                for j in range(i + 1, len(A)):
                    A[k][j] += m * A[i][j]
                b[k] += m * b[i]
    for i in range (0, len(A)):
        b[i] /= A[i][i]
    ans = np.zeros((len(A), 1))
    fin_order = np.zeros(len(A))
    for i in range(0, len(A)):
        fin_order[x_order[i]] = i
    for i in range(0, len(A)):
        ans[i] = b[int(fin_order[i])]
    print(np.matmul(A_0, ans) - b_0)

[[-1.42108547e-13]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 2.84217094e-14]
 [ 0.00000000e+00]
 [-1.42108547e-14]
 [ 0.00000000e+00]
 [-1.42108547e-14]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.42108547e-14]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.42108547e-14]
 [ 0.00000000e+00]
 [-1.42108547e-14]
 [ 0.00000000e+00]
 [ 1.42108547e-14]
 [ 1.42108547e-14]
 [ 0.00000000e+00]
 [ 1.42108547e-14]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.42108547e-14]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 7.10542736e-15]
 [-7.10542736e-15]
 [ 7.10542736e-15]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 7.10542736e-15]
 [-7.10542736e-15]
 [ 1.42108547e-14]
 [ 7.10542736e-15]
 [ 7.10542736e-15]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.0000000

### Метод верхней релаксиции ###

In [5]:
def norm_1(vector):
	res = 0.0
	for i in range(vector.shape[0]):
		val = np.abs(vector[i])
		if val > res:
			res = val
	return res

def norm_2(vector):
	res = 0.0
	for i in range(vector.shape[0]):
		res += np.abs(vector[i])
	return res

def norm_3(vector):
	res = np.sqrt(np.dot(vector, vector))
	return res

def matr_norm_1(matrix):
    lin = matrix.shape[0]
    col = matrix.shape[1]
    norm = 0.0
    for i in range(lin):
        new_norm = 0.0
        for j in range(col):
            new_norm += np.abs(matrix[i, j])
        if new_norm > norm:
            norm = new_norm
    return norm

def matr_norm_2(matrix):
    lin = matrix.shape[0]
    col = matrix.shape[1]
    norm = 0.0
    for i in range(col):
        new_norm = 0.0
        for j in range(lin):
            new_norm += np.abs(matrix[i, j])
        if new_norm > norm:
            norm = new_norm
    return norm

def matr_norm_3(matrix):
	res = np.sqrt(find_max_eigen(np.matmul(matrix.transpose(), matrix)))
	return res

In [6]:
def vec_abs(x, y):
	sum = 0
	for i in range(x.size):
		sum = sum + (x[i] - y[i]) ** 2
	return np.sqrt(sum)

def get_relax_solution(matrix, column, tau):
	n = matrix.shape[0]
	matr_step_B = np.eye(n) - np.dot(matrix, tau)
	matr_step_f = np.dot(column, tau)
	x = np.array([1] * n)
	while True:
		x_start = x
		x = np.matmul(matr_step_B, x_start) + matr_step_f
		if(vec_abs(x_start, x) < precision):
			break
	return x

def diverg(matrix, column, solution, f):
	return f(np.matmul(matrix, solution) - column[:, 0])

def condition_num(matrix, f):
	return f(matrix) * f(np.linalg.inv(matrix))

def condition_numbers(matrix):
	print("Condition number:")
	print("Norm №1 ", condition_num(matrix, matr_norm_1))
	print("Norm №2 ", condition_num(matrix, matr_norm_2))
	print("Norm №3 ", condition_num(matrix, matr_norm_3))
	
def matrix_diffs(matrix, relax_solut):
	print("Matrix diff:")
	print("Norm №1 ", diverg(matrix, b, relax_solut, norm_1))
	print("Norm №2 ", diverg(matrix, b, relax_solut, norm_2))
	print("Norm №3 ", np.around(diverg(matrix, b, relax_solut, norm_3), decimals=5))

In [7]:
lambda_max = find_max_eigen(A)
lambda_min = find_min_eigen(A)
tau = 1 / (lambda_max + lambda_min)

print("Stop criteria is ", precision)
relax_solut = get_relax_solution(A, b[:, 0], tau)

condition_numbers(A)
matrix_diffs(A, relax_solut)

print("Transposed given matrix b:\n", np.transpose(b))
print("Transposed calculated matrix b:\n", np.around(np.transpose(np.matmul(A, relax_solut))))

Stop criteria is  1e-06
Condition number:
Norm №1  1114.2539006984796
Norm №2  1114.2539006984796
Norm №3  30.76823343697627
Matrix diff:
Norm №1  1.1261771533099818e-05
Norm №2  1.2671911493455923e-05
Norm №3  1e-05
Transposed given matrix b:
 [[100.  99.  98.  97.  96.  95.  94.  93.  92.  91.  90.  89.  88.  87.
   86.  85.  84.  83.  82.  81.  80.  79.  78.  77.  76.  75.  74.  73.
   72.  71.  70.  69.  68.  67.  66.  65.  64.  63.  62.  61.  60.  59.
   58.  57.  56.  55.  54.  53.  52.  51.  50.  49.  48.  47.  46.  45.
   44.  43.  42.  41.  40.  39.  38.  37.  36.  35.  34.  33.  32.  31.
   30.  29.  28.  27.  26.  25.  24.  23.  22.  21.  20.  19.  18.  17.
   16.  15.  14.  13.  12.  11.  10.   9.   8.   7.   6.   5.   4.   3.
    2.   1.]]
Transposed calculated matrix b:
 [100.  99.  98.  97.  96.  95.  94.  93.  92.  91.  90.  89.  88.  87.
  86.  85.  84.  83.  82.  81.  80.  79.  78.  77.  76.  75.  74.  73.
  72.  71.  70.  69.  68.  67.  66.  65.  64.  63.  62.  61.  